# 🧠 Agentic Analytics with CrewAI: A Minimal Working Example

This notebook demonstrates a simplified **agentic data pipeline** using [CrewAI](https://github.com/joaomdmoura/crewai). It showcases how multiple AI agents can work together to perform real-world analytics tasks—in this case, loading and visualizing the Iris dataset.

We simulate:
- 🛰️ A **Data Ingestion Agent**
- 📊 A **Visualization Agent**

These agents collaborate in a structured pipeline, allowing you to experience how agent roles and task delegation work in practice—**without glue code or manual orchestration**.

---

👤 **Author**: [Dr. K.M. Mohsin](https://www.linkedin.com/in/km-mohsin/)  
Follow me on LinkedIn to explore more around **Agentic AI, CrewAI, LangGraph, and real-world LLM applications in analytics, automation, and education.**

## ✅ Step 1: Install Dependencies and LLM Key

In [ ]:
!pip install -U crewai openai pandas matplotlib tabulate python-dotenv

### Place .env file and save your LLM Key

In [ ]:
# Load your OpenAI API key securely
import os
from dotenv import load_dotenv

# Option 1 (Quick, not secure)
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Option 2 (Preferred: .env file with OPENAI_API_KEY=sk-...)
load_dotenv()


## 🧠 Step 2: Define Tools using `@tool`

In [ ]:

from crewai.tools import tool

@tool("Load Iris Dataset")
def load_data_tool(_input: str) -> str:
    """Loads the Iris dataset from a CSV file."""
    import pandas as pd
    df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")
    return df.to_json()

@tool("Generate Scatter Plot")
def plot_data_tool(df_json: str) -> str:
    """Generates a scatter plot of sepal length vs petal length."""
    import pandas as pd
    import matplotlib.pyplot as plt
    df = pd.read_json(df_json)
    df.plot(kind='scatter', x='sepal_length', y='petal_length')
    plt.title("Sepal vs Petal Length")
    plt.savefig("plot.png")
    return "Plot saved to plot.png"


## 👥 Step 3: Define Agents

In [ ]:
from crewai import Agent

def build_agents():
    # Added 'goal' and 'backstory' parameters to the Agent initializations
    ingestion = Agent(
        name="IngestionAgent",
        role="Fetches data",
        goal="Load the Iris dataset from a specified URL and provide it for further processing.",
        backstory="This agent is responsible for accessing external data sources and retrieving the necessary information for analysis.",
        tools=[load_data_tool]
    )
    visualization = Agent(
        name="VizAgent",
        role="Plots data",
        goal="Generate a scatter plot using the provided data, specifically focusing on sepal length versus petal length.",
        backstory="This agent specializes in data visualization, taking structured data and creating informative plots.",
        tools=[plot_data_tool]
    )
    return ingestion, visualization


## 🛠️ Step 4: Run the Crew

In [ ]:

from crewai import Crew
from crewai import Task

ingestion_agent, viz_agent = build_agents()

crew = Crew(
    agents=[ingestion_agent, viz_agent],
    tasks=[
        Task(
            agent=ingestion_agent,
            description="Load the Iris dataset from the specified URL for visualization.",
            expected_output="A JSON string representing the Iris dataset.",
        ),
        Task(
            agent=viz_agent,
            description="Generate a scatter plot of sepal length versus petal length from the provided Iris dataset.",
            expected_output="A file named 'plot.png' containing the scatter plot.",
        )
    ]
)

crew.kickoff()